In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
import numpy as np

## DATA CLEANING AND TIDYING (For visualization and EDA )

1. First let's read the dataset using Pandas's read_csv function and see the shape of it (observations and features) and it's columns and potential problems like missing values, duplicates and check the data constistency(whether a numeric feature is indeed in an numeric datatype).

In [6]:
telecom_data = pd.read_csv("Telecom Customers Churn.csv")

In [7]:
telecom_data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [8]:
telecom_data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

2. I will transform these column names into snake_case for easier usage.

In [10]:
telecom_data.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

3. Apparently no null or missing values

In [12]:
telecom_data.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

4. All looks okay except for TotalCharges which is of type object which makes me think there is a null value or something that is not encoded right so I will look into it

In [14]:
telecom_data.duplicated().sum()

0

5. No duplicates are apparent.
6. I will now create a functions to convert strings to snake_case and another function to aplly that conversion on all columns in the DataFrame

In [16]:
def convert_to_snake_case(column_name):
    """
    Convert a string to snake_case.

    Parameters:
    column_name (str): The original column name.

    Returns:
    str: The column name converted to snake_case.
    """
  
    snake_case_name = re.sub(r'[\W\s]+', '_', column_name)
    
   
    snake_case_name = re.sub(r'(?<!^)(?=[A-Z])', '_', snake_case_name)
    

    snake_case_name = snake_case_name.lower()
    
   
    snake_case_name = re.sub(r'_+', '_', snake_case_name)
    
   
    snake_case_name = snake_case_name.strip('_')
    
    return snake_case_name

def rename_columns_to_snake_case(df):
    """
    Rename all columns in a DataFrame to snake_case.

    Parameters:
    df (pd.DataFrame): The DataFrame whose columns need to be renamed.

    Returns:
    pd.DataFrame: A new DataFrame with columns renamed to snake_case.
    """
    
    column_mapping = {col: convert_to_snake_case(col) for col in df.columns}
    
    
    df_snake_case = df.rename(columns=column_mapping)
    
    return df_snake_case

In [17]:
telecom_customer_data_snake_case = rename_columns_to_snake_case(telecom_data)

7. Let's see the result

In [19]:
telecom_customer_data_snake_case

,customer_i_d,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [20]:
telecom_customer_data_snake_case.columns

Index(['customer_i_d', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'internet_service',
       'online_security', 'online_backup', 'device_protection', 'tech_support',
       'streaming_t_v', 'streaming_movies', 'contract', 'paperless_billing',
       'payment_method', 'monthly_charges', 'total_charges', 'churn'],
      dtype='object')

8. Let's see the unique values to gather some understanding why is the total_charges and object column and see the others
9. I will create a function to display the unique values and their count.

In [22]:
def display_unique_values_and_counts(df):
    """
    Display unique values and their counts for each column in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The DataFrame whose columns will be analyzed.
    
    Returns:
    None
    """
    for column in df.columns:
        print(f"Column: {column}")
        print(df[column].value_counts(dropna = False))
        print("-" * 50)  


display_unique_values_and_counts(telecom_customer_data_snake_case)

Column: customer_i_d
customer_i_d
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64
--------------------------------------------------
Column: gender
gender
Male      3555
Female    3488
Name: count, dtype: int64
--------------------------------------------------
Column: senior_citizen
senior_citizen
0    5901
1    1142
Name: count, dtype: int64
--------------------------------------------------
Column: partner
partner
No     3641
Yes    3402
Name: count, dtype: int64
--------------------------------------------------
Column: dependents
dependents
No     4933
Yes    2110
Name: count, dtype: int64
--------------------------------------------------
Column: tenure
tenure
1     613
72    362
2     238
3     200
4     176
     ... 
28     57
39     56
44     51
36     50
0      11
Name: count, Length: 73, dtype: int64
---------

10. Here we can clearly see that 11 of the total_charge entries are blank that's why it doesn't count them as null, but they arent numbers either, so I shall remove them, but let's make sure 

In [24]:
telecom_customer_data_snake_case['total_charges'] = pd.to_numeric(telecom_customer_data_snake_case['total_charges'], errors='coerce')
total_charges_issues = telecom_customer_data_snake_case['total_charges'].isnull().sum()
print(f"Number of invalid 'TotalCharges' entries: {total_charges_issues}")

Number of invalid 'TotalCharges' entries: 11


11. Let's drop those entries

In [26]:
telecom_customer_data_cleaned = telecom_customer_data_snake_case.dropna(subset=['total_charges'])

In [27]:
telecom_customer_data_cleaned 

,customer_i_d,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,...,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


12. Let's remove the customer_i_d column as it doesn't provide any valuable information 

In [29]:
telecom_data_cleaned = telecom_customer_data_cleaned.drop(['customer_i_d'], axis=1)

13. Let's save the cleaned data in new csv 

In [31]:
telecom_data_cleaned.to_csv("test_cleaned.csv", index = False)

14. Last check to see if all is good in the new file 

In [33]:
new_data = pd.read_csv('test_cleaned.csv')

In [34]:
new_data

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [35]:
new_data.isnull().sum()

gender               0
senior_citizen       0
partner              0
dependents           0
tenure               0
phone_service        0
multiple_lines       0
internet_service     0
online_security      0
online_backup        0
device_protection    0
tech_support         0
streaming_t_v        0
streaming_movies     0
contract             0
paperless_billing    0
payment_method       0
monthly_charges      0
total_charges        0
churn                0
dtype: int64

In [36]:
new_data.dtypes

gender                object
senior_citizen         int64
partner               object
dependents            object
tenure                 int64
phone_service         object
multiple_lines        object
internet_service      object
online_security       object
online_backup         object
device_protection     object
tech_support          object
streaming_t_v         object
streaming_movies      object
contract              object
paperless_billing     object
payment_method        object
monthly_charges      float64
total_charges        float64
churn                 object
dtype: object

In [37]:
new_data.duplicated().sum()

22

15. Here we see duplicates so let's remove them

In [39]:
telecom_data_cleaned_no_duplicates = new_data.drop_duplicates()

In [40]:
telecom_data_cleaned_no_duplicates

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [41]:
telecom_data_cleaned_no_duplicates.reset_index(drop=True, inplace=True)

In [42]:
telecom_data_cleaned_no_duplicates

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7006,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7007,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7008,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


In [43]:
telecom_data_cleaned_no_duplicates.duplicated().sum()

0

16. This data is now clean and ready to be visualized and explored in detail. No null values, no duplicates, consistant data types and easily usable column names.
17. Let's now create a new csv file before we continue with preparing it for machine learning deployment

In [45]:
telecom_data_cleaned_no_duplicates.to_csv("cleaned_data_telecom.csv",index=False)

## Data preparing for ML algorithm

18. Okay now let's continue by preparing the data for machine learning deployment , beggining with one-hot encoding as it would work better than manually mapping the categories to values like 0,1,2 will introduce a FALSE sense of order in places where there ain't no order ("None" < "DSL" < "Fiber optic") which is not the case they are seperate things and do not follow any natural order and the model could assume false relationships.
19. One-hot encoding ensures that all categories are treated equally by the model, without implying any rank or order between categories. This is especially important for models like logistic regression or neural networks.
20. For that we will have to convert the categorical columns into dummies which in this case I assume would come with the addition of new features, as the encoding works with (True(1) and False(0))and in some columns we have 3 values.

In [48]:
telecom_data_cleaned_no_duplicates

,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_t_v,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7006,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7007,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7008,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


21. Let's map the categorical columns so that we can easily use them in the conversion

In [50]:
categorical_columns = [
    'gender', 'partner', 'dependents', 'phone_service', 'multiple_lines',
    'internet_service', 'online_security', 'online_backup', 'device_protection',
    'tech_support', 'streaming_t_v', 'streaming_movies', 'contract',
    'paperless_billing', 'payment_method', 'churn'
]

22. We will get the dummies of these categorical columns which will give us a boolean of True and False or 1 and 0 

In [52]:
telecom_data_encoded = pd.get_dummies(telecom_data_cleaned_no_duplicates, columns=categorical_columns, drop_first=True)

In [53]:
telecom_data_encoded

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,multiple_lines_Yes,...,streaming_t_v_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_One year,contract_Two year,paperless_billing_Yes,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,churn_Yes
0,0,1,29.85,29.85,False,True,False,False,True,False,...,False,False,False,False,False,True,False,True,False,False
1,0,34,56.95,1889.50,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
2,0,2,53.85,108.15,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,True,True
3,0,45,42.30,1840.75,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
4,0,2,70.70,151.65,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,0,24,84.80,1990.50,True,True,True,True,False,True,...,True,False,True,True,False,True,False,False,True,False
7006,0,72,103.20,7362.90,False,True,True,True,False,True,...,True,False,True,True,False,True,True,False,False,False
7007,0,11,29.60,346.45,False,True,True,False,True,False,...,False,False,False,False,False,True,False,True,False,False
7008,1,4,74.40,306.60,True,True,False,True,False,True,...,False,False,False,False,False,True,False,False,True,True


23. As we can see it clearly added new features and that's normal as there are multiple categorical variables with multiple categories.
24. For each categorical variable, one-hot encoding creates a new binary column for each unique category (minus one to avoid multicollinearity, if drop_first=True is used).
25. For example, if you have a categorical variable like contract with three categories: "Month-to-month," "One year," and "Two year," one-hot encoding will add two new columns (contract_One year and contract_Two year), and it will imply that if both are 0, the contract is "Month-to-month."
26. Yes increasing the number of features will lead to longer process of training, especially if it's an SVM(Support Vector Machine), but for that we can do Feature selection and not use the features that have low or no impact on the model or a posteriori do a Dimensionality Reduction. But we will get back to that when we start model training and validation.
27. Now another thing we should do beforehand is SCALING the numerical values as it's recommended to use for Gradient-based algorithms like logistic regression, neural networks, support vector machines. These algorithms are sensitive to feature magnitudes. Without scaling, features with larger ranges (e.g., total_charges) can dominate the optimization process, leading to poor model performance.

In [55]:
numerical_columns = ['tenure', 'monthly_charges', 'total_charges']

28. We will be using StandardScaler for scaling the numerical values

In [57]:
scaler = StandardScaler()

In [58]:
telecom_data_encoded[numerical_columns] = scaler.fit_transform(telecom_data_encoded[numerical_columns])

In [59]:
telecom_data_encoded

,senior_citizen,tenure,monthly_charges,total_charges,gender_Male,partner_Yes,dependents_Yes,phone_service_Yes,multiple_lines_No phone service,multiple_lines_Yes,...,streaming_t_v_Yes,streaming_movies_No internet service,streaming_movies_Yes,contract_One year,contract_Two year,paperless_billing_Yes,payment_method_Credit card (automatic),payment_method_Electronic check,payment_method_Mailed check,churn_Yes
0,0,-1.285566,-1.165523,-0.997284,False,True,False,False,True,False,...,False,False,False,False,False,True,False,True,False,False
1,0,0.060346,-0.264071,-0.176848,True,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,False
2,0,-1.244781,-0.367189,-0.962740,True,False,False,True,False,False,...,False,False,False,False,False,True,False,False,True,True
3,0,0.508983,-0.751387,-0.198355,True,False,False,False,True,False,...,False,False,False,True,False,False,False,False,False,False
4,0,-1.244781,0.193308,-0.943549,False,False,False,True,False,False,...,False,False,False,False,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7005,0,-0.347506,0.662329,-0.132289,True,True,True,True,False,True,...,True,False,True,True,False,True,False,False,True,False
7006,0,1.610184,1.274384,2.237896,False,True,True,True,False,True,...,True,False,True,True,False,True,True,False,False,False
7007,0,-0.877714,-1.173839,-0.857607,False,True,True,False,True,False,...,False,False,False,False,False,True,False,True,False,False
7008,1,-1.163210,0.316384,-0.875188,True,True,False,True,False,True,...,False,False,False,False,False,True,False,False,True,True


29. Now let's save this into a new csv and then proceed with feature selection validation and removing the churn as its TARGET FEATURE and so on..... IF i am gonna proceed with that project in general\

In [61]:
telecom_data_encoded.to_csv("telecom_data_cleaned_encoded.csv", index=False)

30. Let's split the data into test an train sets

In [63]:
X = telecom_data_encoded.drop('churn_Yes', axis=1)
y = telecom_data_encoded['churn_Yes']

In [64]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)


svm_clf = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
svm_clf.fit(X_train, y_train)


y_pred_val = svm_clf.predict(X_val)
print(classification_report(y_val, y_pred_val))
print(f"SVM AUC-ROC on Validation Set: {roc_auc_score(y_val, y_pred_val):.4f}")


              precision    recall  f1-score   support

       False       0.91      0.76      0.83      1014
        True       0.56      0.81      0.66       388

    accuracy                           0.77      1402
   macro avg       0.73      0.78      0.74      1402
weighted avg       0.81      0.77      0.78      1402

SVM AUC-ROC on Validation Set: 0.7816


In [65]:
comparison_df = pd.DataFrame({
    'Actual': y_val,
    'Predicted': y_pred_val
})
print(comparison_df.head(20)) 


      Actual  Predicted
665     True       True
5841   False      False
5070   False       True
4012   False      False
1937   False      False
2953    True      False
6925   False      False
1612   False       True
5128   False      False
6230   False      False
5182   False      False
5358   False      False
3018    True       True
1259   False      False
4843    True       True
1040   False      False
2895    True       True
1354    True       True
2508   False       True
4158   False      False


## 16 out of 20/ I will be working on diff models and parameters tomorrow

In [67]:
data1 = pd.read_csv('new_data_cleaned_1.csv')

In [68]:
data2 = pd.read_csv('new_data_cleaned_2.csv')

In [69]:
data1_snake = rename_columns_to_snake_case(data1)

In [70]:
data1_snake

,call_failure,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,frequency_of_s_m_s,distinct_called_numbers,age_group,tariff_plan,status,age,customer_value,f_n,f_p,churn
0,8,0,38,0,4370,71,5,17,3,1,1,30,197.640,177.8760,69.7640,0
1,0,0,39,0,318,5,7,4,2,1,2,25,46.035,41.4315,60.0000,0
2,10,0,37,0,2453,60,359,24,3,1,1,30,1536.520,1382.8680,203.6520,0
3,10,0,38,0,4198,66,1,35,1,1,1,15,240.020,216.0180,74.0020,0
4,3,0,38,0,2393,58,2,33,1,1,1,15,145.805,131.2245,64.5805,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,17,0,17,1,9237,177,80,42,5,1,1,55,261.210,235.0890,76.1210,0
2846,13,0,18,4,3157,51,38,21,3,1,1,30,280.320,252.2880,78.0320,0
2847,8,1,11,2,1792,25,7,9,3,1,1,30,100.680,90.6120,60.0680,1
2848,26,0,38,9,10740,99,8,44,5,2,1,55,174.585,157.1265,67.4585,0


In [71]:
data1_snake.columns

Index(['call_failure', 'complains', 'subscription_length', 'charge_amount',
       'seconds_of_use', 'frequency_of_use', 'frequency_of_s_m_s',
       'distinct_called_numbers', 'age_group', 'tariff_plan', 'status', 'age',
       'customer_value', 'f_n', 'f_p', 'churn'],
      dtype='object')

In [72]:
numerical_columns_1 = [ 
  'call_failure', 'complains', 'subscription_length', 'charge_amount',
       'seconds_of_use', 'frequency_of_use', 'frequency_of_s_m_s',
       'distinct_called_numbers', 'age_group', 'tariff_plan', 'status', 'age',
       'customer_value', 'f_n', 'f_p'
]

In [73]:
data1_snake[numerical_columns_1] = scaler.fit_transform(data1_snake[numerical_columns_1])

In [74]:
data1_snake

,call_failure,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,frequency_of_s_m_s,distinct_called_numbers,age_group,tariff_plan,status,age,customer_value,f_n,f_p,churn
0,0.026969,-0.296287,0.636013,-0.628722,-0.039115,0.008975,-0.613961,-0.399660,0.184601,-0.295586,-0.561951,-0.121574,-0.539223,-0.539223,-0.570354,0
1,-1.065198,-0.296287,0.750672,-0.628722,-1.004112,-1.141022,-0.596110,-1.155874,-0.934786,-0.295586,1.779513,-0.685884,-0.833972,-0.833972,-0.763535,0
2,0.300011,-0.296287,0.521355,-0.628722,-0.495655,-0.182691,2.545548,0.007532,0.184601,-0.295586,-0.561951,-0.121574,2.063820,2.063820,2.078632,0
3,0.300011,-0.296287,0.636013,-0.628722,-0.080077,-0.078146,-0.649661,0.647405,-2.054174,-0.295586,-0.561951,-1.814504,-0.456828,-0.456828,-0.486504,0
4,-0.655636,-0.296287,0.636013,-0.628722,-0.509944,-0.217540,-0.640736,0.531064,-2.054174,-0.295586,-0.561951,-1.814504,-0.640000,-0.640000,-0.672910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,1.255657,-0.296287,-1.771817,0.016295,1.119977,1.855940,0.055427,1.054596,2.423375,-0.295586,-0.561951,2.699975,-0.415631,-0.415631,-0.444580,0
2846,0.709573,-0.296287,-1.657159,1.951347,-0.327995,-0.339509,-0.319430,-0.166979,0.184601,-0.295586,-0.561951,-0.121574,-0.378477,-0.378477,-0.406770,0
2847,0.026969,3.375101,-2.459769,0.661313,-0.653074,-0.792538,-0.596110,-0.865023,0.184601,-0.295586,-0.561951,-0.121574,-0.727732,-0.727732,-0.762190,1
2848,2.484345,-0.296287,0.636013,5.176434,1.477922,0.496853,-0.587185,1.170937,2.423375,3.383107,-0.561951,2.699975,-0.584046,-0.584046,-0.615968,0


In [75]:
data2

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,SC,79,415,No,No,0,134.7,98,22.90,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,False
2662,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
2663,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
2664,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False


In [76]:
data2_snake = rename_columns_to_snake_case(data2)

In [77]:
data2_snake

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,SC,79,415,No,No,0,134.7,98,22.90,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,False
2662,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
2663,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
2664,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False


In [78]:
data2_no_state = data2_snake.drop(['state'], axis=1)

In [79]:
data2_no_state

,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,79,415,No,No,0,134.7,98,22.90,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,False
2662,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
2663,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
2664,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False


In [80]:
data2_no_state.columns

Index(['account_length', 'area_code', 'international_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'customer_service_calls', 'churn'],
      dtype='object')

In [81]:
categorical_columns_2 = [ 
    'international_plan', 'voice_mail_plan',
]

In [139]:
data2_encoded = pd.get_dummies(data2_no_state, columns=categorical_columns_2, drop_first=True)

In [141]:
data2_encoded.to_csv('encoded_data_2.csv', index = False)

In [83]:
numerical_columns_2 = [ 
  'account_length', 'area_code',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'customer_service_calls',
]

In [84]:
data2_encoded[numerical_columns_2] = scaler.fit_transform(data2_encoded[numerical_columns_2])

In [85]:
data2_encoded

,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn,international_plan_Yes,voice_mail_plan_Yes
0,0.692163,-0.527811,1.247508,1.579670,0.484868,1.579942,-0.058619,-0.050781,-0.058445,0.857403,-0.469031,0.856707,-0.085020,-0.597527,-0.085681,-0.429172,False,False,True
1,0.161278,-0.527811,1.320985,-0.329918,1.135375,-0.330194,-0.095916,0.147654,-0.095397,1.048458,0.149054,1.049293,1.242179,-0.597527,1.242921,-0.429172,False,False,True
2,0.919686,-0.527811,-0.589414,1.179302,0.685024,1.179465,-1.554439,0.494917,-1.554963,-0.759668,0.200561,-0.758391,0.704125,0.216894,0.698194,-1.191955,False,False,False
3,-0.420168,-0.692467,-0.589414,2.212509,-1.466653,2.212675,-2.718509,-0.596479,-2.718922,-0.084083,-0.572045,-0.084339,-1.304609,1.031314,-1.307995,0.333610,False,True,False
4,-0.647691,-0.527811,-0.589414,-0.235822,0.634985,-0.235772,-1.022461,1.090224,-1.021482,-0.281046,1.076181,-0.281303,-0.049150,-0.597527,-0.045823,1.096392,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,-0.546569,-0.527811,-0.589414,-0.826226,-0.115600,-0.826178,-0.209771,-1.588658,-0.210869,0.398478,1.436730,0.397126,0.560644,0.216894,0.565334,0.333610,False,False,False
2662,2.310100,-0.527811,2.055754,-0.429549,-1.166419,-0.430042,0.296687,1.288660,0.297208,1.534958,-0.881087,1.535136,-0.120891,0.624104,-0.125539,0.333610,False,False,True
2663,-0.824652,-0.527811,-0.589414,0.952365,-2.167199,0.952636,-0.922347,-2.233574,-0.922176,-0.194382,1.179195,-0.193763,-0.228501,-0.190317,-0.231828,1.096392,False,False,False
2664,-1.835863,1.706797,-0.589414,0.024324,0.434829,0.024701,1.735580,-2.084747,1.735990,-0.182564,-0.469031,-0.180632,1.385660,0.624104,1.389067,0.333610,False,False,False


In [86]:
X_2 = data2_encoded.drop(columns=['churn'])
y_2 = data2_encoded['churn']

In [87]:
data1_snake

,call_failure,complains,subscription_length,charge_amount,seconds_of_use,frequency_of_use,frequency_of_s_m_s,distinct_called_numbers,age_group,tariff_plan,status,age,customer_value,f_n,f_p,churn
0,0.026969,-0.296287,0.636013,-0.628722,-0.039115,0.008975,-0.613961,-0.399660,0.184601,-0.295586,-0.561951,-0.121574,-0.539223,-0.539223,-0.570354,0
1,-1.065198,-0.296287,0.750672,-0.628722,-1.004112,-1.141022,-0.596110,-1.155874,-0.934786,-0.295586,1.779513,-0.685884,-0.833972,-0.833972,-0.763535,0
2,0.300011,-0.296287,0.521355,-0.628722,-0.495655,-0.182691,2.545548,0.007532,0.184601,-0.295586,-0.561951,-0.121574,2.063820,2.063820,2.078632,0
3,0.300011,-0.296287,0.636013,-0.628722,-0.080077,-0.078146,-0.649661,0.647405,-2.054174,-0.295586,-0.561951,-1.814504,-0.456828,-0.456828,-0.486504,0
4,-0.655636,-0.296287,0.636013,-0.628722,-0.509944,-0.217540,-0.640736,0.531064,-2.054174,-0.295586,-0.561951,-1.814504,-0.640000,-0.640000,-0.672910,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2845,1.255657,-0.296287,-1.771817,0.016295,1.119977,1.855940,0.055427,1.054596,2.423375,-0.295586,-0.561951,2.699975,-0.415631,-0.415631,-0.444580,0
2846,0.709573,-0.296287,-1.657159,1.951347,-0.327995,-0.339509,-0.319430,-0.166979,0.184601,-0.295586,-0.561951,-0.121574,-0.378477,-0.378477,-0.406770,0
2847,0.026969,3.375101,-2.459769,0.661313,-0.653074,-0.792538,-0.596110,-0.865023,0.184601,-0.295586,-0.561951,-0.121574,-0.727732,-0.727732,-0.762190,1
2848,2.484345,-0.296287,0.636013,5.176434,1.477922,0.496853,-0.587185,1.170937,2.423375,3.383107,-0.561951,2.699975,-0.584046,-0.584046,-0.615968,0


In [88]:
X_3 = data1_snake.drop(columns=['churn'])
y_3 = data1_snake['churn']

In [89]:
X_train_val_2, X_test_2, y_train_val_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.2, random_state=42)
X_train_2, X_val_2, y_train_2, y_val_2 = train_test_split(X_train_val_2, y_train_val_2, test_size=0.25, random_state=42)

In [90]:
y_3 = y_3.astype(int)

# Split data into training/validation and test sets
X_train_val_3, X_test_3, y_train_val_3, y_test_3 = train_test_split(X_3, y_3, test_size=0.2, random_state=42)

# Split the training data further into training and validation sets
X_train_3, X_val_3, y_train_3, y_val_3 = train_test_split(X_train_val_3, y_train_val_3, test_size=0.25, random_state=42)

svm_clf = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
svm_clf.fit(X_train_3, y_train_3)


y_val_pred_3 = svm_clf.predict(X_val_3)
print(classification_report(y_val_3, y_val_pred_3))
print(f"SVM AUC-ROC on Validation Set: {roc_auc_score(y_val_3, y_val_pred_3):.4f}")

              precision    recall  f1-score   support

           0       0.98      0.84      0.90       468
           1       0.56      0.91      0.69       102

    accuracy                           0.85       570
   macro avg       0.77      0.88      0.80       570
weighted avg       0.90      0.85      0.87       570

SVM AUC-ROC on Validation Set: 0.8768


In [91]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

svm_clf = SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42)
svm_clf.fit(X_train_2, y_train_2)


y_val_pred_2 = svm_clf.predict(X_val_2)
print(classification_report(y_val_2, y_val_pred_2))
print(f"SVM AUC-ROC on Validation Set: {roc_auc_score(y_val, y_pred_val):.4f}")

              precision    recall  f1-score   support

       False       0.97      0.90      0.93       474
        True       0.49      0.76      0.60        59

    accuracy                           0.89       533
   macro avg       0.73      0.83      0.76       533
weighted avg       0.92      0.89      0.90       533

SVM AUC-ROC on Validation Set: 0.7816


In [174]:
data1_snake.to_csv("customer_churn_second.csv",index=False)

In [163]:
data2_encoded.to_csv("customer_churn_third.csv",index = False)